In [1]:
from pyspark.sql import functions as sf


In [6]:

from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType, TimestampNTZType, MapType

schema = StructType([
    StructField("_source",StructType([
        StructField("customer",StringType()),
        StructField("customerID",StringType()),
        StructField("eventId",StringType()),
        StructField("eventTime",TimestampType()),
        StructField("metricTimestamp",TimestampType()),        
        StructField("domain",StringType()),
        StructField("resource",StringType()),
        StructField("metricName",StringType()),
        StructField("metricValue",DoubleType()),
        StructField("deviceName",StringType()),
        StructField("destDeviceName",StringType()),
        StructField("interfaceName",StringType()),
        StructField("destInterfaceName",StringType()),
        ]))
    ])





In [ ]:
%%sql

DROP TABLE nyc.meraki_latency;


ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '/': extra input '/'.(line 3, pos 0)

== SQL ==

DROP TABLE nyc.meraki_31_latency;
//DROP TABLE nyc.meraki_latency;
^^^


In [16]:
%%sql

DROP TABLE nyc.meraki_31_latency;


++
||
++
++

In [7]:
df2 = spark.read.schema(schema).json("s3a://ingress/cisco-meraki-tmf688-metric-latency_data.31.jsonl.gz");
#df2 = spark.read.schema(schema).json("s3a://ingress/cisco-meraki-tmf688-metric-latency_data.34.jsonl.gz");
#df2 = spark.read.schema(schema).json("s3a://ingress/cisco-meraki-tmf688-metric-latency_data.51.jsonl.gz");


In [8]:
df2.head()

Row(_source=Row(customer='Belron Germany', customerID='ACCTV0017956', eventId='b1a56627-dda7-419f-a37e-6fc276c41dfd', eventTime=datetime.datetime(2024, 12, 20, 12, 45, 53), metricTimestamp=datetime.datetime(2024, 12, 20, 12, 35, 53), domain='Fixed Connectivity', resource='node=VF1502-7131-GW-P$$interface=wan1(Q2JY-S4K9-QQJ2)$$remoteinterface=wan1(Q2CZ-7N4R-VMUD)$$remotenode=AWS-VMX1-HUB1', metricName='Latency', metricValue=16.0, deviceName='VF1502-7131-GW-P', destDeviceName='AWS-VMX1-HUB1', interfaceName='wan1(Q2JY-S4K9-QQJ2)', destInterfaceName='wan1(Q2CZ-7N4R-VMUD)'))

In [9]:
df2.count()

1095863

In [10]:


df3 = df2.select(df2._source.customer.alias("customer"), 
           df2._source.eventId.alias("eventId"),
           df2._source.metricTimestamp.alias("eventTime"),           
           df2._source.eventTime.alias("timeReceived"),
           df2._source.resource.alias("source_id"),
           sf.create_map([
               sf.lit("customerID"),        df2._source.customerID,                
           ]).alias("source_labels"),
           df2._source.metricName.alias("metric_name"),
           df2._source.metricValue.alias("metric_value"),
           sf.create_map([
               sf.lit("deviceName"),        df2._source.deviceName, 
               sf.lit("destDeviceName"),    df2._source.destDeviceName,
               sf.lit("interfaceName"),     df2._source.interfaceName, 
               sf.lit("destInterfaceName"), df2._source.destInterfaceName,               
           ]).alias("metric_labels")
          )



In [11]:
df3.head()

Row(customer='Belron Germany', eventId='b1a56627-dda7-419f-a37e-6fc276c41dfd', eventTime=datetime.datetime(2024, 12, 20, 12, 35, 53), timeReceived=datetime.datetime(2024, 12, 20, 12, 45, 53), source_id='node=VF1502-7131-GW-P$$interface=wan1(Q2JY-S4K9-QQJ2)$$remoteinterface=wan1(Q2CZ-7N4R-VMUD)$$remotenode=AWS-VMX1-HUB1', source_labels={'customerID': 'ACCTV0017956'}, metric_name='Latency', metric_value=16.0, metric_labels={'destDeviceName': 'AWS-VMX1-HUB1', 'interfaceName': 'wan1(Q2JY-S4K9-QQJ2)', 'deviceName': 'VF1502-7131-GW-P', 'destInterfaceName': 'wan1(Q2CZ-7N4R-VMUD)'})

In [17]:
df3.write.saveAsTable("nyc.meraki_31_latency")
#df3.write.insertInto("nyc.meraki_latency")


In [18]:
%%sql

DESCRIBE TABLE nyc.meraki_latency;

col_name,data_type,comment
customer,string,None
eventId,string,None
eventTime,timestamp,None
timeReceived,timestamp,None
source_id,string,None
source_labels,"map<string,string>",None
metric_name,string,None
metric_value,double,None
metric_labels,"map<string,string>",None


In [19]:
%%sql

SELECT source_id,source_labels from nyc.meraki_latency limit 20;

source_id,source_labels
node=VF1502-7131-GW-P$$interface=wan1(Q2JY-S4K9-QQJ2)$$remoteinterface=wan1(Q2CZ-7N4R-VMUD)$$remotenode=AWS-VMX1-HUB1,{'customerID': 'ACCTV0017956'}
node=VF1502-7131-GW-P$$interface=wan1(Q2JY-S4K9-QQJ2)$$remoteinterface=wan1(Q2CZ-7N4R-VMUD)$$remotenode=AWS-VMX1-HUB1,{'customerID': 'ACCTV0017956'}
node=VF1502-7131-GW-P$$interface=wan1(Q2JY-S4K9-QQJ2)$$remoteinterface=wan1(Q2CZ-7N4R-VMUD)$$remotenode=AWS-VMX1-HUB1,{'customerID': 'ACCTV0017956'}
node=VF1502-7131-GW-P$$interface=wan1(Q2JY-S4K9-QQJ2)$$remoteinterface=wan1(Q2CZ-Z37T-BGKT)$$remotenode=AWS-VMX2-HUB2,{'customerID': 'ACCTV0017956'}
node=VF1502-7131-GW-P$$interface=wan1(Q2JY-S4K9-QQJ2)$$remoteinterface=wan1(Q2CZ-Z37T-BGKT)$$remotenode=AWS-VMX2-HUB2,{'customerID': 'ACCTV0017956'}
node=VF1502-7131-GW-P$$interface=wan1(Q2JY-S4K9-QQJ2)$$remoteinterface=wan1(Q2CZ-Z37T-BGKT)$$remotenode=AWS-VMX2-HUB2,{'customerID': 'ACCTV0017956'}
node=VF1502-7331-GW-P$$interface=wan1(Q2JY-D4GY-3RW5)$$remoteinterface=wan1(Q2CZ-7N4R-VMUD)$$remotenode=AWS-VMX1-HUB1,{'customerID': 'ACCTV0017956'}
node=VF1502-7331-GW-P$$interface=wan1(Q2JY-D4GY-3RW5)$$remoteinterface=wan1(Q2CZ-7N4R-VMUD)$$remotenode=AWS-VMX1-HUB1,{'customerID': 'ACCTV0017956'}
node=VF1502-7331-GW-P$$interface=wan1(Q2JY-D4GY-3RW5)$$remoteinterface=wan1(Q2CZ-7N4R-VMUD)$$remotenode=AWS-VMX1-HUB1,{'customerID': 'ACCTV0017956'}
node=VF1502-7331-GW-P$$interface=wan1(Q2JY-D4GY-3RW5)$$remoteinterface=wan1(Q2CZ-Z37T-BGKT)$$remotenode=AWS-VMX2-HUB2,{'customerID': 'ACCTV0017956'}


In [20]:
%%sql

select count(*) from nyc.meraki_latency;

count(1)
1095863
